In [1]:
!python3 mnist_train.py

Directory  /home/anubhav/Desktop/MNIST_DNNDK_lab/tb_logs created 
Directory  /home/anubhav/Desktop/MNIST_DNNDK_lab/chkpts created 
Epoch 1 / 3
 Step:    0  Training accuracy: 0.1092
 Step:  100  Training accuracy: 0.8422
 Step:  200  Training accuracy: 0.8653
 Step:  300  Training accuracy: 0.9026
 Step:  400  Training accuracy: 0.9197
 Step:  500  Training accuracy: 0.9310
 Step:  600  Training accuracy: 0.9301
 Step:  700  Training accuracy: 0.9411
 Step:  800  Training accuracy: 0.9407
 Step:  900  Training accuracy: 0.9504
 Step: 1000  Training accuracy: 0.9506
Epoch 2 / 3
 Step:    0  Training accuracy: 0.9559
 Step:  100  Training accuracy: 0.9586
 Step:  200  Training accuracy: 0.9564
 Step:  300  Training accuracy: 0.9632
 Step:  400  Training accuracy: 0.9650
 Step:  500  Training accuracy: 0.9663
 Step:  600  Training accuracy: 0.9657
 Step:  700  Training accuracy: 0.9694
 Step:  800  Training accuracy: 0.9660
 Step:  900  Training accuracy: 0.9702
 Step: 1000  Training accu

In [2]:
!rm -rf ./freeze
!mkdir ./freeze

!freeze_graph --input_graph=./chkpts/training_graph.pb \
             --input_checkpoint=./chkpts/float_model.ckpt \
             --input_binary=true \
             --output_graph=./freeze/frozen_graph.pb \
             --output_node_names=dense_1/BiasAdd


/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anubhav/anaconda3/envs/decent/lib/pyt

In [3]:
!python3 eval_graph.py \
  --graph ./freeze/frozen_graph.pb \
  --input_node images_in \
  --output_node dense_1/BiasAdd \
  --gpu 0

Instructions for updating:
Use tf.gfile.GFile.
 Top 1 accuracy with validation set: 0.9785
 Top 5 accuracy with validation set: 0.9998
FINISHED!


In [4]:
!python3 generate_images.py
!rm -rf ./quantize_results
!decent_q quantize \
  --input_frozen_graph ./freeze/frozen_graph.pb \
  --input_nodes images_in \
  --input_shapes ?,28,28,1 \
  --output_nodes dense_1/BiasAdd \
  --method 1 \
  --input_fn graph_input_fn.calib_input \
  --gpu 0 \
  --calib_iter 200

Directory /home/anubhav/Desktop/MNIST_DNNDK_lab/calib_dir created
FINISHED GENERATING CALIBRATION IMAGES
/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '

#####################################
EVALUATE QUANTIZED GRAPH
#####################################
Instructions for updating:
Use tf.gfile.GFile.
2021-09-23 02:40:27.863604: W tensorflow/core/framework/allocator.cc:122] Allocation of 432640000 exceeds 10% of system memory.
2021-09-23 02:40:28.415027: W tensorflow/core/framework/allocator.cc:122] Allocation of 432640000 exceeds 10% of system memory.
2021-09-23 02:40:29.662564: W tensorflow/core/framework/allocator.cc:122] Allocation of 108160000 exceeds 10% of system memory.
2021-09-23 02:40:29.747572: W tensorflow/core/framework/allocator.cc:122] Allocation of 108160000 exceeds 10% of system memory.
2021-09-23 02:40:30.054557: W tensorflow/core/framework/allocator.cc:122] Allocation of 154880000 exceeds 10% of system memory.
 Top 1 accuracy with validation set: 0.9801
 Top 5 accuracy with validation set: 0.9999
FINISHED!
#####################################
EVALUATE QUANTIZED GRAPH COMPLETED
#####################################


### Generate dcf

In [6]:
!dlet -f ./pynqz2_dpu.hwh
!mv -v ./*.dcf ./pynqz2_dpu.dcf

[DLet]Generate DPU DCF file dpu-04-15-2021-17-15.dcf successfully.
'./dpu-04-15-2021-17-15.dcf' -> './pynqz2_dpu.dcf'


In [7]:
!rm -rf ./compile
!mkdir ./compile
!dnnc \
       --parser=tensorflow \
       --frozen_pb=./quantize_results/deploy_model.pb \
       --dcf=pynqz2_dpu.dcf \
       --cpu_arch=arm32 \
       --output_dir=compile \
       --save_kernel \
       --mode=normal \
       --net_name=mnist


DNNC Kernel topology "mnist_kernel_graph.jpg" for network "mnist"
DNNC kernel list info for network "mnist"
                               Kernel ID : Name
                                       0 : mnist

                             Kernel Name : mnist
--------------------------------------------------------------------------------
                             Kernel Type : DPUKernel
                               Code Size : 7.49KB
                              Param Size : 0.95MB
                           Workload MACs : 3.30MOPS
                         IO Memory Space : 7.31KB
                              Mean Value : 0, 0, 0, 
                              Node Count : 4
                            Tensor Count : 5
                    Input Node(s)(H*W*C)
                        conv2d_Conv2D(0) : 28*28*1
                   Output Node(s)(H*W*C)
                       dense_1_MatMul(0) : 1*1*10




In [8]:
!decent_q inspect --input_frozen_graph ./quantize_results/deploy_model.pb

/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anubhav/anaconda3/envs/decent/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anubhav/anaconda3/envs/decent/lib/pyt